In [110]:
import numpy as np

In [111]:
WIDTH_CHAMER = 7
LEFT_DISTANCE = 2
HEIGHT_ABOVE_GROUND = 3
N_ROCKS_FALL = 2022

In [112]:
with open('17_rocks.txt') as f:
    lines = f.readlines()

In [113]:
rocks = []

rock = []
for line in lines:
    if line.strip() == '':
        rocks.append(np.vstack(rock))
        rock = []
    else:
        line = line.strip().replace('#', '@')
        line = [x for x in line]
        for _ in range(LEFT_DISTANCE):
            line.insert(0, '.')
        while len(line) < WIDTH_CHAMER:
            line.append('.')
        rock.append(line)
rocks.append(np.vstack(rock))
        

In [114]:
with open('17.txt') as f:
    lines = f.readlines()

In [115]:
for line in lines:
    line = line.strip()
    line = [x for x in line]
jets = line

In [116]:
test = [2,3]

In [117]:
test.pop(0)

2

In [118]:
len(test)

1

In [146]:
cave = np.empty((1, WIDTH_CHAMER), dtype=object)
cave_space = np.empty((HEIGHT_ABOVE_GROUND, WIDTH_CHAMER), dtype=object)
rocks_yield = rocks.copy()
jet_yield = jets.copy()
cave_space[:] = '.'
cave[:] = '#'


def check_if_falling_rock_is_touching_ground(cave):
    for y, x in zip(*np.where(cave == '@')):
        if cave[y+1, x] == '#':
            return True
    
    return False

def remove_only_air_rows(cave):
    mask = np.all(cave == '.', axis=1)
    return cave[~mask, :].copy()

def move_all_rocks(cave, direction):
    cave_before = cave.copy()
    falling_rocks_idx = list(zip(*np.where(cave == '@')))
    falling_rocks_x = np.where(cave == '@')[1]
    if direction == 'v':
        falling_rocks_idx.reverse() # To get the bottom ones first.
        for y, x in falling_rocks_idx:
            cave[y+1, x] = '@'
            cave[y, x] = '.'
    elif direction == '>':
        falling_rocks_idx.reverse()
        if not np.max(falling_rocks_x) == 6:
            for y, x in falling_rocks_idx:
                if cave[y, x+1] == '#':
                    return cave_before
                cave[y, x+1] = '@'
                cave[y, x] = '.'
    elif direction == '<':
        if not np.min(falling_rocks_x) == 0:
            for y, x in falling_rocks_idx:
                if cave[y, x-1] == '#':
                    return cave_before
                cave[y, x-1] = '@'
                cave[y, x] = '.'
            
    return cave
 
from tqdm import tqdm
height_deltas = []
for i in tqdm(range(1, 5000+1)):
    rock = rocks_yield.pop(0)
    if len(rocks_yield) == 0:
        rocks_yield = rocks.copy()
    cave_before = cave.copy()
    cave = np.vstack((rock, cave_space, cave))
    nr_jets_until_rock_hits_ground = 0
    while True:
        jet = jet_yield.pop(0)
        if len(jet_yield) == 0:
            jet_yield = jets.copy()
        # Get all falling rocks @
        cave = move_all_rocks(cave, jet)
        # Check if falling rock is touching ground
        if check_if_falling_rock_is_touching_ground(cave):
            cave[cave == '@'] = '#'
            cave = remove_only_air_rows(cave)
            height_deltas.append(cave.shape[0] - cave_before.shape[0])
            break
        else:
            cave = move_all_rocks(cave, 'v')


100%|██████████| 5000/5000 [00:45<00:00, 110.97it/s]


In [159]:
def get_pattern(seq):
    seq2 = seq
    outs = {}
    l = 0
    r = 0
    c = None

    for end in range(len(seq2)+1):
      for start in range(end):
          word = chr(1).join(seq2[start:end])
          if not word in outs:
              outs[word] = 1
          else:
              outs[word] += 1
    for item in outs:
        if outs[item] > r or (len(item) > l and outs[item] > 1):
            l = len(item)
            r = outs[item]
            c = item
    return c.split(chr(1))

In [207]:
# Python 3 program to find the longest repeated
# non-overlapping substring

# Returns the longest repeating non-overlapping
# substring in str
# stolen from geeksforgeeks
def longestRepeatedSubstring(str):

	n = len(str)
	LCSRe = [[0 for x in range(n + 1)]
				for y in range(n + 1)]

	res = "" # To store result
	res_length = 0 # To store length of result

	# building table in bottom-up manner
	index = 0
	for i in range(1, n + 1):
		for j in range(i + 1, n + 1):
			
			# (j-i) > LCSRe[i-1][j-1] to remove
			# overlapping
			if (str[i - 1] == str[j - 1] and
				LCSRe[i - 1][j - 1] < (j - i)):
				LCSRe[i][j] = LCSRe[i - 1][j - 1] + 1

				# updating maximum length of the
				# substring and updating the finishing
				# index of the suffix
				if (LCSRe[i][j] > res_length):
					res_length = LCSRe[i][j]
					index = max(i, index)
					start_i = i
				
			else:
				LCSRe[i][j] = 0

	# If we have non-empty result, then insert
	# all characters from first character to
	# last character of string
	if (res_length > 0):
		for i in range(index - res_length + 1,
									index + 1):
			res = res + str[i - 1]

	return start_i-len(res), [int(x) for x in res] # Somehow, i gives the end of the pattern, so I subtract the length of the pattern to get the start.


In [208]:
max(height_deltas)

4

In [209]:
idx_pattern_start, longest_pattern = longestRepeatedSubstring(''.join([str(x) for x in height_deltas]))

In [225]:
# All github copilot, nothing me.
nr_patterns_appear = (1000000000000 - idx_pattern_start) // len(longest_pattern)
nr_rocks_after_pattern = 1000000000000 - idx_pattern_start - nr_patterns_appear * len(longest_pattern)

581395348

In [227]:
# All github copilot, nothing me.
sum(height_deltas[:idx_pattern_start]) \
    + sum(longest_pattern) * nr_patterns_appear \
        +sum(height_deltas[idx_pattern_start:idx_pattern_start+nr_rocks_after_pattern])

1572093023267